In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
# %matplotlib inline
# Set up TensorBoard
writer = SummaryWriter('runs/cifar100_resnet18')
writer_1 = SummaryWriter('runs/cifar100_transformer')
class config:
    seed = 42
    epoch = 100
    best = 0
    learning_rate = 0.001
    batch_size = 100
    num_workers = 2
    num_classes = 100
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


trainset = dataset.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=config.batch_size,
                                          shuffle=True, num_workers=2)

# Load the test set
testset = dataset.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)

testloader = DataLoader(testset, batch_size=config.batch_size,
                                         shuffle=False, num_workers=2)

# The classes
classes = trainset.classes


100%|██████████| 169001437/169001437 [00:11<00:00, 15117379.83it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, num_classes=100):
        super(ResNet, self).__init__()
        expansion=3
        self.conv1 = nn.Conv2d(3, 16*expansion, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16*expansion)

        self.layer1 = self._make_layer(16*expansion, 16*expansion, stride=1)
        self.layer2 = self._make_layer(16*expansion, 32*expansion, stride=2)
        self.layer3 = self._make_layer(32*expansion, 64*expansion, stride=2)
        
        self.linear = nn.Linear(1024*expansion, num_classes)

    def _make_layer(self, in_channels, out_channels, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, 2):  # you can adjust the range for depth control
            layers.append(ResidualBlock(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
#         print(out.shape)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
#         print(out.shape)
        out = nn.functional.avg_pool2d(out, 2)
#         print(out.shape)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# check the total parameters
model = ResNet(num_classes=100).to(config.device)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total parameters:', total_params)
# from torchsummary import summary
# summary(model,(3,32,32),batch_size=1,device="cuda")

Total parameters: 1869460


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)


def train(epoch):
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(config.device))
        loss = criterion(outputs, labels.to(config.device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i%240 ==0:
            print(f'loss: {running_loss}')
        # log the running loss
        writer.add_scalar('training loss',
                        running_loss,
                        epoch * len(trainloader) + i)
        
        running_loss = 0.0

    print(f"Finished Training Epoch {epoch}")


def test(epoch):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images.to(config.device))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted.to('cpu') == labels).sum().item()
        acc = correct / total
        # log the test accuracy
        writer.add_scalar('test accuracy',
                        acc,
                        epoch)
    if acc>config.best:
        config.best=acc
        torch.save(model,'best.pth')

    print(f'best test acc: {config.best*100}')
    print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}")


In [5]:
for epoch in tqdm(range(100)):
    print(epoch+1)
    train(epoch)
    test(epoch)

print('Finished Training')

# After all the logging is done, close the SummaryWriter
writer.close()


  0%|          | 0/100 [00:00<?, ?it/s]

1


0it [00:00, ?it/s]

loss: 4.673364162445068
loss: 3.438734531402588
loss: 3.00478458404541
Finished Training Epoch 0
best test acc: 30.4
Accuracy of the network on the 10000 test images: 30.4
2


0it [00:00, ?it/s]

...

loss: 0.02081446349620819
loss: 0.02367299050092697
loss: 0.004302026238292456
Finished Training Epoch 97
best test acc: 57.809999999999995
Accuracy of the network on the 10000 test images: 54.98
99


0it [00:00, ?it/s]

loss: 0.002931439084932208
loss: 0.0020032613538205624
loss: 0.0012021794682368636
Finished Training Epoch 98
best test acc: 57.809999999999995
Accuracy of the network on the 10000 test images: 56.07
100


0it [00:00, ?it/s]

loss: 0.001505729858763516
loss: 0.00037520835758186877
loss: 0.00024107882927637547
Finished Training Epoch 99
best test acc: 57.809999999999995
Accuracy of the network on the 10000 test images: 56.59
Finished Training
